#SQL Injection Detection - ROOT CAUSE ANALYSIS (using ML)

#Importing Libraries

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install tensorflow-hub
!pip install tfds-nightly

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
import sklearn
from sklearn.utils import shuffle
from sklearn import preprocessing , neighbors , svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import GradientBoostingClassifier

import keras
from keras import layers
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

from nltk import ngrams
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
import numpy as np
import pandas as pd
import glob
import time
from xml.dom import minidom
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import math
import collections
import re
from itertools import groupby

In [ ]:
print("Tensorflow Version: ", tf.__version__)
print("Hub version: ", hub.__version__)
print("Keras version: ", keras.__version__)
print("Nltk version: ", nltk.__version__)
print("Sklearn version: ", sklearn.__version__)
print("Pandas version: ", pd.__version__)

Tensorflow Version:  2.5.0
Hub version:  0.12.0
Keras version:  2.5.0
Nltk version:  3.2.5
Sklearn version:  0.22.2.post1
Pandas version:  1.1.5


#Loading Datasets

In [ ]:
dfSQLI = pd.read_csv("sqli.csv", encoding='utf-16') 
sql = pd.read_csv('sql.csv')
password = pd.read_csv('password.csv')
username = pd.read_csv('username.csv')
sqli = pd.read_csv('sqli4.csv')
username.dropna(axis=0, how="all", inplace=True)
password.dropna(axis=0, how="all", inplace=True)
username.reset_index(drop=True, inplace=True)
password.reset_index(drop=True, inplace=True)

In [ ]:
vectorizer = CountVectorizer(min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(dfSQLI['Sentence'].values.astype('U')).toarray()
transformed_posts = pd.DataFrame(posts)

In [ ]:
dfSQLI = pd.concat([dfSQLI,transformed_posts], axis=1)
X_df = dfSQLI[dfSQLI.columns[2:]]
y_df = dfSQLI['Label']

#Fill "LENGTH" attribute as per corresponding rows

In [ ]:
def leng(df,col,len_col):
  for i in range(len(df)):
    cl = df[col][i]
    length = len(str(cl))
    df[len_col][i] = length
  return df
  
username = leng(username, 'Query','Length')
password = leng(password, 'Query','Length')
sql = leng(sql, 'Query','Length')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


#Compile DataFrame

In [ ]:
username['Label'] = 'username'
password['Label'] = 'password'
sqli['Label'] = 'sqli'
sql['Label'] = 'sql'

sqli.drop(['Attack'], axis=1, inplace=True)
username.drop(['Attack'], axis=1, inplace=True)
password.drop(['Attack'], axis=1, inplace=True)
sql.drop(['Attack'], axis=1, inplace=True)

df = pd.concat([sqli,sql,username,password])
df.reset_index(drop=True,inplace=True)

In [ ]:
df

,Query,Length,Label
0,1' where 6406=6406;select count(*) from rdb$fi...,115.0,sqli
1,1) and 8514=(select count(*) from domain.domai...,111.0,sqli
2,-3136%') or 3400=6002,21.0,sqli
3,1) where 7956=7956 or sleep(5)#,31.0,sqli
4,-7387'))) order by 1--,22.0,sqli
...,...,...,...
12882,1978,4.0,password
12883,1011980,7.0,password
12884,wildcat,7.0,password
12885,polina,6.0,password


#Calculate punctuation in SQL query for each row

In [ ]:
def cal_puncndop(df,col,punop_col,l):
  df1 = df[[col]].copy()
  for i, query in enumerate(df[col]):
    count = 0
    li = list(query)
    for ch in range(len(query)):
      if query[ch] in l:
        li [ch] = " "
        #df[col][i] = "".join(li)
        count = count+1
    df1[col][i] = "".join(li)
    #print(("".join(li)))
    df[punop_col][i] = count
  df[col] = df1[col]
  return df  

df['punctuation'] = 0
#df['operator'] = 0
df = cal_puncndop(df,'Query','punctuation',
                  ['!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?","[","]",")","("])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]




```
'''
df=cal_puncndop(df,'Query','operator',
                 ["<",">", "<=", ">=", "=", "==", "!=", "<<", ">>", "|", "&", "-", "+", "%", "^", "*"]) 
                 should * be included becoz count(*) !!!
'''
```





In [ ]:
df

,Query,Length,Label,punctuation
0,1 where 6406=6406 select count * from rdb$fi...,115.0,sqli,9
1,1 and 8514= select count * from domain domai...,111.0,sqli,11
2,3136% or 3400=6002,21.0,sqli,3
3,1 where 7956=7956 or sleep 5 #,31.0,sqli,3
4,7387 order by 1,22.0,sqli,7
...,...,...,...,...
12882,1978,4.0,password,0
12883,1011980,7.0,password,0
12884,wildcat,7.0,password,0
12885,polina,6.0,password,0


#Create tokens for SQL keywords



```
TOKENS:

"(?P<UNION>UNION\s+(ALL\s+)?SELECT)|(?P<PREFIX>([\'\"\)]|"+ 
"((\'|\"|\)|\d+|\w+)\s))(\|\|\&\&|and|or|as|where|IN\sBOOLEAN\sMODE)"+
"(\s|\()(\(?\'?-?\d+\'?(=|LIKE|<|>|<=|>=)\'?-?\d+|\(?[\'\"\\\"]\S+"[\'\"\\\"]"+"(\s+)?(=|LIKE|<|>|<=|>=)(\s+)?[\'\"\\\"]))|"+
"(?P<USUAL>([\'\"]\s*)(\|\||\&\&|and|or)"+"(\s*[\'\"])(\s*[\'\"])=)|"+
"(?P<DROP>;\s*DROP\s+(TABLE|DATABASE)\s(IF\s+EXISTS\s)?\S+)|+ 
"(?P<NOTIN>\snot\sin\s?\((\d+|(\'|\")\w+(\'|\"))\))|"+
"(?P<LIMIT>LIMIT\s+\d+(\s+)?,(\s+)?\d+)|GROUP_CONCAT\((?P<GRPCONCAT>.*?)\)|"+
"(?P<ORDERBY>ORDER\s+BY\s+\d+)|CONCAT\((?P<CONCAT>.*?)\)|"+
"(?P<CASEWHEN>\(CASE\s(\d+\s|\(\d+=\d+\)\s|NULL\s)?WHEN\s(\d+|\(?\d+=\d+\)?|NULL)\sTHEN\s(\d+|\(\d+=\d+\)|NULL)\sELSE)|"+
"(?P<DBNAME>(?:(?:m(?:s(?:ysaccessobjects|ysaces|ysobjects|ysqueries|ysrelationships|ysaccessstorage|ysaccessxml|ysmodules|ysmodules2|db)|aster\.\.sysdatabases|ysql\.db)|"+
"s(?:ys(?:\.database_name|aux)|chema(?:\W*\(|_name)|qlite(_temp)?_master)|+"
"d(?:atabas|b_nam)e\W*\(|information_schema|pg_(catalog|toast)|northwind|tempdb)))|"+
"(?P<DATABASE>DATABASE\(\))|(?P<DTCNAME>table_name|column_name|table_schema|schema_name)|"+
"(?P<CAST>CAST\(.*AS\s+\w+\))|(?P<INQUERY>\(SELECT[^a-z_0-9])|(?P<CHRBYPASS>((CHA?R\(\d+\)(,|\|\||\+)\s?)+)|"+
"CHA?R\((\d+,\s?)+\))|(?P<FROMDB>\sfrom\s(dual|sysmaster|sysibm)[\s.:])|"+
"(?P<MYSQLFUNC>[^.](ABS|ACOS|ADDDATE|ADDTIME|AES_DECRYPT|AES_ENCRYPT|ANY_VALUE|ASCII|ASIN|ASYMMETRIC_DECRYPT|"+
"ASYMMETRIC_DERIVE|ASYMMETRIC_ENCRYPT|ASYMMETRIC_SIGN|ASYMMETRIC_VERIFY|ATAN|ATAN2|AVG|BENCHMARK|BIN|BIT_AND|BIT_COUNT|BIT_LENGTH|BIT_OR|BIT_XOR|"+
"CAST|CEIL|CEILING|CHAR|CHAR_LENGTH|CHARACTER_LENGTH|CHARSET|COALESCE|COERCIBILITY|COLLATION|COMPRESS|CONCAT|CONCAT_WS|CONNECTION_ID|"+
"CONV|CONVERT|CONVERT_TZ|COS|COT|COUNT|COUNT|CRC32|CREATE_ASYMMETRIC_PRIV_KEY|CREATE_ASYMMETRIC_PUB_KEY|CREATE_DH_PARAMETERS|CREATE_DIGEST|"+
"CURDATE|CURRENT_DATE|CURRENT_TIME|CURRENT_TIMESTAMP|CURRENT_USER|CURTIME|DATABASE|DATE|DATE_ADD|DATE_FORMAT|DATE_SUB|DATEDIFF|DAY|DAYNAME|"+
"DAYOFMONTH|DAYOFWEEK|DAYOFYEAR|DECODE|DEFAULT|DEGREES|ELT|ENCODE|EXP|EXPORT_SET|EXTRACT|EXTRACTVALUE|FIELD|FIND_IN_SET|FLOOR|FORMAT|FOUND_ROWS|"+
"FROM_BASE64|FROM_DAYS|FROM_UNIXTIME|GeometryCollection|GET_FORMAT|GET_LOCK|GREATEST|GROUP_CONCAT|GTID_SUBSET|GTID_SUBTRACT|HEX|HOUR|IF|IFNULL|IIF|"+
"IN|INET_ATON|INET_NTOA|INET6_ATON|INET6_NTOA|INSERT|INSTR|INTERVAL|IS_FREE_LOCK|IS_IPV4|IS_IPV4_COMPAT|IS_IPV4_MAPPED|IS_IPV6|IS_USED_LOCK|ISNULL|"+
"JSON_APPEND|JSON_ARRAY|JSON_ARRAY_APPEND|JSON_ARRAY_INSERT|JSON_CONTAINS|JSON_CONTAINS_PATH|JSON_DEPTH|JSON_EXTRACT|JSON_INSERT|JSON_KEYS|JSON_LENGTH|+"
"JSON_MERGE|JSON_OBJECT|JSON_QUOTE|JSON_REMOVE|JSON_REPLACE|JSON_SEARCH|JSON_SET|JSON_TYPE|JSON_UNQUOTE|JSON_VALID|LAST_INSERT_ID|LCASE|LEAST|LEFT|LENGTH|"+
"LineString|LN|LOAD_FILE|LOCALTIME|LOCALTIMESTAMP|LOCATE|LOG|LOG10|LOG2|LOWER|LPAD|LTRIM|MAKE_SET|MAKEDATE|MAKETIME|MASTER_POS_WAIT|MAX|MBRContains|MBRCoveredBy|"+
"MBRCovers|MBRDisjoint|MBREquals|MBRIntersects|MBROverlaps|MBRTouches|MBRWithin|MICROSECOND|MID|MIN|MINUTE|MOD|MONTH|MONTHNAME|MultiLineString|MultiPoint|"+
"MultiPolygon|NAME_CONST|NOT IN|NOW|NULLIF|OCT|OCTET_LENGTH|OLD_PASSWORD|ORD|PERIOD_ADD|PERIOD_DIFF|PI|Point|Polygon|POSITION|POW|POWER|PROCEDURE ANALYSE|"+
"QUARTER|QUOTE|RADIANS|RAND|RANDOM_BYTES|RELEASE_ALL_LOCKS|RELEASE_LOCK|REPEAT|REPLACE|REVERSE|RIGHT|ROUND|ROW_COUNT|RPAD|RTRIM|SCHEMA|SEC_TO_TIME|SECOND|"+
"SESSION_USER|SHA1|SHA2|SIGN|SIN|SLEEP|SOUNDEX|SPACE|SQRT|ST_Area|ST_AsBinary|ST_AsGeoJSON|ST_AsText|ST_Buffer|ST_Buffer_Strategy|ST_Centroid|ST_Contains|"+
"ST_ConvexHull|ST_Crosses|ST_Difference|ST_Dimension|ST_Disjoint|ST_Distance|ST_Distance_Sphere|ST_EndPoint|ST_Envelope|ST_Equals|ST_ExteriorRing|ST_GeoHash|ST_GeomCollFromText|"+
"ST_GeomCollFromWKB|ST_GeometryN|ST_GeometryType|ST_GeomFromGeoJSON|ST_GeomFromText|ST_GeomFromWKB|ST_InteriorRingN|ST_Intersection|ST_Intersects|ST_IsClosed|"+
"ST_IsEmpty|ST_IsSimple|ST_IsValid|ST_LatFromGeoHash|ST_Length|ST_LineFromText|ST_LineFromWKB|ST_LongFromGeoHash|ST_MakeEnvelope|ST_MLineFromText|ST_MLineFromWKB|"+
"ST_MPointFromText|ST_MPointFromWKB|ST_MPolyFromText|ST_MPolyFromWKB|ST_NumGeometries|ST_NumInteriorRing|ST_NumPoints|ST_Overlaps|ST_PointFromGeoHash|ST_PointFromText|"+
"ST_PointFromWKB|ST_PointN|ST_PolyFromText|ST_PolyFromWKB|ST_Simplify|ST_SRID|ST_StartPoint|ST_SymDifference|ST_Touches|ST_Union|ST_Validate|ST_Within|ST_X|ST_Y|"+
"StartPoint|STD|STDDEV|STDDEV_POP|STDDEV_SAMP|STR_TO_DATE|STRCMP|SUBDATE|SUBSTR|SUBSTRING|SUBSTRING_INDEX|SUBTIME|SUM|SYSDATE|SYSTEM_USER|TAN|TIME|TIME_FORMAT|"+
"TIME_TO_SEC|TIMEDIFF|TIMESTAMP|TIMESTAMPADD|TIMESTAMPDIFF|TO_BASE64|TO_DAYS|TO_SECONDS|TRIM|TRUNCATE|UCASE|UNCOMPRESS|UNCOMPRESSED_LENGTH|UNHEX|UNIX_TIMESTAMP|+"
"UpdateXML|UPPER|USER|UTC_DATE|UTC_TIME|UTC_TIMESTAMP|UUID|UUID_SHORT|VALIDATE_PASSWORD_STRENGTH|VALUES|VAR_POP|VAR_SAMP|VARIANCE|VERSION|WAIT_FOR_EXECUTED_GTID_SET|"+
"WAIT_UNTIL_SQL_THREAD_AFTER_GTIDS|WEEK|WEEKDAY|WEEKOFYEAR|WEIGHT_STRING|YEAR|YEARWEEK)\()|"+
"(?P<BOOLEAN>\'?-?\d+\'?(=|LIKE)\'?-?\d+($|\s|\)|,|--|#)|[\'\"\\\"]\S+[\'\"\\\"](\s+)?(=|LIKE)(\s+)?[\'\"\\\"]\S+)|"+
"(?P<PLAIN>(@|##|#)[A-Z]\w+|[A-Z]\w*(?=\s*\.)|(?<=\.)[A-Z]\w*|[A-Z]\w*(?=\()|`(``|[^`])*`|´(´´|[^´])*´|[_A-Z][_$#\w]*|[가-힣]+)"
```



In [ ]:
#make tokens using regex
sql_regex = re.compile("(?P<UNION>UNION\s+(ALL\s+)?SELECT)|(?P<PREFIX>([\'\"\)]|((\'|\"|\)|\d+|\w+)\s))(\|\|\&\&|and|or|as|where|IN\sBOOLEAN\sMODE)(\s|\()(\(?\'?-?\d+\'?(=|LIKE|<|>|<=|>=)\'?-?\d+|\(?[\'\"\\\"]\S+[\'\"\\\"](\s+)?(=|LIKE|<|>|<=|>=)(\s+)?[\'\"\\\"]))|(?P<USUAL>([\'\"]\s*)(\|\||\&\&|and|or)(\s*[\'\"])(\s*[\'\"])=)|(?P<DROP>;\s*DROP\s+(TABLE|DATABASE)\s(IF\s+EXISTS\s)?\S+)|(?P<NOTIN>\snot\sin\s?\((\d+|(\'|\")\w+(\'|\"))\))|(?P<LIMIT>LIMIT\s+\d+(\s+)?,(\s+)?\d+)|GROUP_CONCAT\((?P<GRPCONCAT>.?)\)|(?P<ORDERBY>ORDER\s+BY\s+\d+)|CONCAT\((?P<CONCAT>.?)\)|(?P<CASEWHEN>\(CASE\s(\d+\s|\(\d+=\d+\)\s|NULL\s)?WHEN\s(\d+|\(?\d+=\d+\)?|NULL)\sTHEN\s(\d+|\(\d+=\d+\)|NULL)\sELSE)|(?P<DBNAME>(?:(?:m(?:s(?:ysaccessobjects|ysaces|ysobjects|ysqueries|ysrelationships|ysaccessstorage|ysaccessxml|ysmodules|ysmodules2|db)|aster\.\.sysdatabases|ysql\.db)|s(?:ys(?:\.database_name|aux)|chema(?:\W*\(|name)|qlite(_temp)?_master)|d(?:atabas|b_nam)e\W*\(|information_schema|pg(catalog|toast)|northwind|tempdb)))|(?P<DATABASE>DATABASE\(\))|(?P<DTCNAME>table_name|column_name|table_schema|schema_name)|(?P<CAST>CAST\(.AS\s+\w+\))|(?P<INQUERY>\(SELECT[^a-z_0-9])|(?P<CHRBYPASS>((CHA?R\(\d+\)(,|\|\||\+)\s?)+)|CHA?R\((\d+,\s?)+\))|(?P<FROMDB>\sfrom\s(dual|sysmaster|sysibm)[\s.:])|(?P<MYSQLFUNC>[^.](ABS|ACOS|ADDDATE|ADDTIME|AES_DECRYPT|AES_ENCRYPT|ANY_VALUE|ASCII|ASIN|ASYMMETRIC_DECRYPT|ASYMMETRIC_DERIVE|ASYMMETRIC_ENCRYPT|ASYMMETRIC_SIGN|ASYMMETRIC_VERIFY|ATAN|ATAN2|AVG|BENCHMARK|BIN|BIT_AND|BIT_COUNT|BIT_LENGTH|BIT_OR|BIT_XOR|CAST|CEIL|CEILING|CHAR|CHAR_LENGTH|CHARACTER_LENGTH|CHARSET|COALESCE|COERCIBILITY|COLLATION|COMPRESS|CONCAT|CONCAT_WS|CONNECTION_ID|CONV|CONVERT|CONVERT_TZ|COS|COT|COUNT|COUNT|CRC32|CREATE_ASYMMETRIC_PRIV_KEY|CREATE_ASYMMETRIC_PUB_KEY|CREATE_DH_PARAMETERS|CREATE_DIGEST|CURDATE|CURRENT_DATE|CURRENT_TIME|CURRENT_TIMESTAMP|CURRENT_USER|CURTIME|DATABASE|DATE|DATE_ADD|DATE_FORMAT|DATE_SUB|DATEDIFF|DAY|DAYNAME|DAYOFMONTH|DAYOFWEEK|DAYOFYEAR|DECODE|DEFAULT|DEGREES|ELT|ENCODE|EXP|EXPORT_SET|EXTRACT|EXTRACTVALUE|FIELD|FIND_IN_SET|FLOOR|FORMAT|FOUND_ROWS|FROM_BASE64|FROM_DAYS|FROM_UNIXTIME|GeometryCollection|GET_FORMAT|GET_LOCK|GREATEST|GROUP_CONCAT|GTID_SUBSET|GTID_SUBTRACT|HEX|HOUR|IF|IFNULL|IIF|IN|INET_ATON|INET_NTOA|INET6_ATON|INET6_NTOA|INSERT|INSTR|INTERVAL|IS_FREE_LOCK|IS_IPV4|IS_IPV4_COMPAT|IS_IPV4_MAPPED|IS_IPV6|IS_USED_LOCK|ISNULL|JSON_APPEND|JSON_ARRAY|JSON_ARRAY_APPEND|JSON_ARRAY_INSERT|JSON_CONTAINS|JSON_CONTAINS_PATH|JSON_DEPTH|JSON_EXTRACT|JSON_INSERT|JSON_KEYS|JSON_LENGTH|JSON_MERGE|JSON_OBJECT|JSON_QUOTE|JSON_REMOVE|JSON_REPLACE|JSON_SEARCH|JSON_SET|JSON_TYPE|JSON_UNQUOTE|JSON_VALID|LAST_INSERT_ID|LCASE|LEAST|LEFT|LENGTH|LineString|LN|LOAD_FILE|LOCALTIME|LOCALTIMESTAMP|LOCATE|LOG|LOG10|LOG2|LOWER|LPAD|LTRIM|MAKE_SET|MAKEDATE|MAKETIME|MASTER_POS_WAIT|MAX|MBRContains|MBRCoveredBy|MBRCovers|MBRDisjoint|MBREquals|MBRIntersects|MBROverlaps|MBRTouches|MBRWithin|MICROSECOND|MID|MIN|MINUTE|MOD|MONTH|MONTHNAME|MultiLineString|MultiPoint|MultiPolygon|NAME_CONST|NOT IN|NOW|NULLIF|OCT|OCTET_LENGTH|OLD_PASSWORD|ORD|PERIOD_ADD|PERIOD_DIFF|PI|Point|Polygon|POSITION|POW|POWER|PROCEDURE ANALYSE|QUARTER|QUOTE|RADIANS|RAND|RANDOM_BYTES|RELEASE_ALL_LOCKS|RELEASE_LOCK|REPEAT|REPLACE|REVERSE|RIGHT|ROUND|ROW_COUNT|RPAD|RTRIM|SCHEMA|SEC_TO_TIME|SECOND|SESSION_USER|SHA1|SHA2|SIGN|SIN|SLEEP|SOUNDEX|SPACE|SQRT|ST_Area|ST_AsBinary|ST_AsGeoJSON|ST_AsText|ST_Buffer|ST_Buffer_Strategy|ST_Centroid|ST_Contains|ST_ConvexHull|ST_Crosses|ST_Difference|ST_Dimension|ST_Disjoint|ST_Distance|ST_Distance_Sphere|ST_EndPoint|ST_Envelope|ST_Equals|ST_ExteriorRing|ST_GeoHash|ST_GeomCollFromText|ST_GeomCollFromWKB|ST_GeometryN|ST_GeometryType|ST_GeomFromGeoJSON|ST_GeomFromText|ST_GeomFromWKB|ST_InteriorRingN|ST_Intersection|ST_Intersects|ST_IsClosed|ST_IsEmpty|ST_IsSimple|ST_IsValid|ST_LatFromGeoHash|ST_Length|ST_LineFromText|ST_LineFromWKB|ST_LongFromGeoHash|ST_MakeEnvelope|ST_MLineFromText|ST_MLineFromWKB|ST_MPointFromText|ST_MPointFromWKB|ST_MPolyFromText|ST_MPolyFromWKB|ST_NumGeometries|ST_NumInteriorRing|ST_NumPoints|ST_Overlaps|ST_PointFromGeoHash|ST_PointFromText|ST_PointFromWKB|ST_PointN|ST_PolyFromText|ST_PolyFromWKB|ST_Simplify|ST_SRID|ST_StartPoint|ST_SymDifference|ST_Touches|ST_Union|ST_Validate|ST_Within|ST_X|ST_Y|StartPoint|STD|STDDEV|STDDEV_POP|STDDEV_SAMP|STR_TO_DATE|STRCMP|SUBDATE|SUBSTR|SUBSTRING|SUBSTRING_INDEX|SUBTIME|SUM|SYSDATE|SYSTEM_USER|TAN|TIME|TIME_FORMAT|TIME_TO_SEC|TIMEDIFF|TIMESTAMP|TIMESTAMPADD|TIMESTAMPDIFF|TO_BASE64|TO_DAYS|TO_SECONDS|TRIM|TRUNCATE|UCASE|UNCOMPRESS|UNCOMPRESSED_LENGTH|UNHEX|UNIX_TIMESTAMP|UpdateXML|UPPER|USER|UTC_DATE|UTC_TIME|UTC_TIMESTAMP|UUID|UUID_SHORT|VALIDATE_PASSWORD_STRENGTH|VALUES|VAR_POP|VAR_SAMP|VARIANCE|VERSION|WAIT_FOR_EXECUTED_GTID_SET|WAIT_UNTIL_SQL_THREAD_AFTER_GTIDS|WEEK|WEEKDAY|WEEKOFYEAR|WEIGHT_STRING|YEAR|YEARWEEK)\()|(?P<BOOLEAN>\'?-?\d+\'?(=|LIKE)\'?-?\d+($|\s|\)|,|--|#)|[\'\"\\\"]\S+[\'\"\\\"](\s+)?(=|LIKE)(\s+)?[\'\"\\\"]\S+)|(?P<PLAIN>(@|##|#)[A-Z]\w+|[A-Z]\w(?=\s*\.)|(?<=\.)[A-Z]\w*|[A-Z]\w*(?=\()|`(``|[^`])`|´(´´|[^´])´|[A-Z][$#\w]*|[가-힣]+)", re.IGNORECASE)

In [ ]:
#tokenize each query (sqli/plain-text) using regex
#feature extraction function --> 1st try is lexical analysis
def Sql_tokenizer(raw_sql):
  if sql_regex.search(raw_sql):
    return [tok[0] for tok in groupby([match.lastgroup for match in sql_regex.finditer(raw_sql)])]
  else:
    return ['PLAIN']

def GetTokenSeq(token_list, N):
  token_seq = []
  for n in range(0,N):
    token_seq+= zip(*(token_list[i:] for i in range(n+1)))
  return [str(tuple) for tuple in token_seq]

#Calculate keywords in SQL query for each row

In [ ]:
def cal_keyword(df,col,key_col,l):
  for i, query in enumerate(df[col]):
    count = 0
    query = query.lower()
    words = query.split()
    for word in words:
      if word in l:
        count = count+1
    df[key_col][i] = count
  return df

df['keyword'] = 0
df = cal_keyword(df, 'Query', 'keyword', 
                 ["select", "update", "insert", "create", "drop", "alter", "rename", "exec", "order", 
                  "group", "sleep","count","where"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
df

,Query,Length,Label,punctuation,keyword
0,1 where 6406=6406 select count * from rdb$fi...,115.0,sqli,9,3
1,1 and 8514= select count * from domain domai...,111.0,sqli,11,2
2,3136% or 3400=6002,21.0,sqli,3,0
3,1 where 7956=7956 or sleep 5 #,31.0,sqli,3,2
4,7387 order by 1,22.0,sqli,7,1
...,...,...,...,...,...
12882,1978,4.0,password,0,0
12883,1011980,7.0,password,0,0
12884,wildcat,7.0,password,0,0
12885,polina,6.0,password,0,0


#Calculate the likelihood ratio by defining different metrics

REFERENCS:<br>
https://en.wikipedia.org/wiki/G-test <br>
https://en.wikipedia.org/wiki/Chi-squared_test

In [ ]:
# G-Test Score for likelihood ratio stats
def G_test_score(count, expected):
  if (count == 0):
    return 0
  else:
    return 2.0 * count * math.log(count/expected)

In [ ]:
# pre processing to get G-Test score
def G_test(tokens, types):
  tokens_cnt = tokens.value_counts().astype(float)
  types_cnt = types.value_counts().astype(float)
  total_cnt = float(sum(tokens_cnt))

  # calculate each token counts
  token_cnt_table = collections.defaultdict(lambda : collections.Counter())
  for _tokens, _types in zip(tokens.values, types.values):
    token_cnt_table[_tokens][_types] += 1

  datax = []  # create dataset using token_CountVal and types -- new dataset in setted by token count, value
  tc_dataframe = pd.DataFrame(list(token_cnt_table.values()), index=token_cnt_table.keys())
  tc_dataframe.fillna(0, inplace=True)

  # calculate expected, g-score
  for column in tc_dataframe.columns.tolist():
    #tc_dataframe[column]+= 1
    tc_dataframe[column+'_exp'] = (tokens_cnt / total_cnt) * types_cnt[column]
    tc_dataframe[column+'_GTest'] = [G_test_score(tkn_count, exp) for tkn_count, exp in 
                                     zip(tc_dataframe[column], tc_dataframe[column+'_exp'])]
  return tc_dataframe

In [ ]:
#string entropy to use another feature- but this may not be sufficient
def Entropy(raw_sql):
  p, lns = collections.Counter(str(raw_sql)), float(len(str(raw_sql)))
  return -sum( count/lns * math.log(count/lns, 2) for count in p.values())

#g-score means of each tokens
def G_means(token_seq, c_name):
  try:
    g_scores = [tc_dataframe.loc[token][c_name] for token in token_seq]
  except KeyError:
    return 0
  return sum(g_scores)/len(g_scores) if g_scores else 0 # Average

#Create Dataset for Queries



```
# read data from gdrive
from google.colab import drive
drive.mount('/content/gdrive')

root_path = '/content/gdrive/MyDrive/trainingdata'
filelist = os.listdir(root_path)
filelist

# upload file as variable_name = plain
from google.colab import files
plain = files.upload()

# convert .txt files into list and df
df_list = []
for file in filelist:
  if file == '.DS_Store':
    continue
  dff = pd.read_csv(os.path.join(root_path,file), sep='Aw3s0meSc0t7', names=['raw_sql'], header=None, engine='python')
  #!--files are .txt, not csv
  df = dff.copy()
  df['type'] = 'plain' if file.split('.')[0] == 'plain' else 'sqli'
  df_list.append(df)
```



In [ ]:
df_list = []

#!--files are .txt, not csv
dfPQuerys = pd.read_csv('plain.txt', sep='Aw3s0meSc0t7', names=['raw_sql'], header=None, engine='python')
dfPQuerys['type'] = 'plain'
df_list.append(dfPQuerys)

#!--files are .txt, not csv
dfSQuerys = pd.read_csv('sql_querys.txt', sep='Aw3s0meSc0t7', names=['raw_sql'], header=None, engine='python')
dfSQuerys['type'] = 'sqli'
df_list.append(dfSQuerys)

In [ ]:
df_list

[                                                raw_sql   type
 0                                   Add plain text here  plain
 1                            “Ne te quaesiveris extra.”  plain
 2           “Man is his own star; and the soul that can  plain
 3                   Render an honest and a perfect man,  plain
 4          Commands all light, all influence, all fate;  plain
 ...                                                 ...    ...
 3689  comment6, http://www.ifreakinglovefishing.com/...  plain
 3690  comment6, http://www.lissycakes.com/mysticism/...  plain
 3691  comment6, http://www.nataliagallo.com/young-ad...  plain
 3692  comment6, http://www.organikjoos.com/trivia/pd...  plain
 3693  comment6, http://www.thejustincredible.com/que...  plain
 
 [3694 rows x 2 columns],
                                                 raw_sql  type
 0     59)) AND 3233 IN (SELECT (CHAR(113)+CHAR(98)+C...  sqli
 1     59)) AND 5376=CONVERT(INT,(SELECT CHAR(113)+CH...  sqli
 2     59")) AN

In [ ]:
# god pandas make to us a dataframe like excel format
dfQuerys = pd.concat(df_list, ignore_index=True)
dfQuerys.dropna(inplace=True) 
print(dfQuerys['type'].value_counts())

sqli     5928
plain    3694
Name: type, dtype: int64


In [ ]:
dfQuerys

,raw_sql,type
0,Add plain text here,plain
1,“Ne te quaesiveris extra.”,plain
2,“Man is his own star; and the soul that can,plain
3,"Render an honest and a perfect man,",plain
4,"Commands all light, all influence, all fate;",plain
...,...,...
9617,;drop databases if exists wordpress,sqli
9618,; DROP DATABASES IF EXIST wordpress,sqli
9619,; DORP DATABASES wordpress,sqli
9620,' or ''='#,sqli


In [ ]:
# tokenize raw sql
dfQuerys['sql_tokens'] = dfQuerys['raw_sql'].map(lambda x: Sql_tokenizer(x))

# get token sequences
dfQuerys['token_seq'] = dfQuerys['sql_tokens'].map(lambda x: GetTokenSeq(x, 3))

_tokens, _types = zip(*[(token,token_type) for token_list,token_type in 
                      zip(dfQuerys['token_seq'], dfQuerys['type']) for token in token_list])
tc_dataframe = G_test(pd.Series(_tokens), pd.Series(_types))

In [ ]:
# now we set real features for machine learning algorithm.
dfQuerys['token_length'] = dfQuerys['sql_tokens'].map(lambda x: len(x))
dfQuerys['entropy'] = dfQuerys['raw_sql'].map(lambda x: Entropy(x))
dfQuerys['sqli_g_means'] = dfQuerys['token_seq'].map(lambda x: G_means(x, 'sqli_GTest'))
dfQuerys['plain_g_means'] = dfQuerys['token_seq'].map(lambda x: G_means(x, 'plain_GTest'))

In [ ]:
dfQuerys

,raw_sql,type,sql_tokens,token_seq,token_length,entropy,sqli_g_means,plain_g_means
0,Add plain text here,plain,[PLAIN],"[('PLAIN',)]",1,3.536887,-5343.643372,11293.651663
1,“Ne te quaesiveris extra.”,plain,[PLAIN],"[('PLAIN',)]",1,3.686419,-5343.643372,11293.651663
2,“Man is his own star; and the soul that can,plain,[PLAIN],"[('PLAIN',)]",1,3.731059,-5343.643372,11293.651663
3,"Render an honest and a perfect man,",plain,[PLAIN],"[('PLAIN',)]",1,3.622739,-5343.643372,11293.651663
4,"Commands all light, all influence, all fate;",plain,[PLAIN],"[('PLAIN',)]",1,3.854223,-5343.643372,11293.651663
...,...,...,...,...,...,...,...,...
9617,;drop databases if exists wordpress,sqli,[PLAIN],"[('PLAIN',)]",1,3.685588,-5343.643372,11293.651663
9618,; DROP DATABASES IF EXIST wordpress,sqli,[PLAIN],"[('PLAIN',)]",1,4.183014,-5343.643372,11293.651663
9619,; DORP DATABASES wordpress,sqli,[PLAIN],"[('PLAIN',)]",1,4.026987,-5343.643372,11293.651663
9620,' or ''='#,sqli,[USUAL],"[('USUAL',)]",1,2.321928,0.157788,0.000000


In [ ]:
# list of feature vectors
features = ['token_length', 'entropy','sqli_g_means','plain_g_means']
XQuerys = dfQuerys[features].to_numpy()
print(type(XQuerys))

<class 'numpy.ndarray'>


In [ ]:
# encode categorical feature
labelencoder_y = LabelEncoder()
yQuerys = labelencoder_y.fit_transform(dfQuerys['type'].tolist())

#Specify "LABEL" for SQL query for each row

In [ ]:
for i,label in enumerate(df['Label']):
  if label in ['sql','username','password']:
    df['Label'][i] = 'non-sqli'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#StandardScaler and LabelEncoding

In [ ]:
def scale_data(df, column_list):
  for column in column_list:
    df[column] = df[column].astype('float')
    encoder = preprocessing.StandardScaler()
    df[column] = encoder.fit_transform(df[column].values.reshape(-1,1))
    print("The ",column, "is encoded")
  return(df)
  
def encode_categorical(df, column_list):
  for column in column_list:
    df[column] = df[column].astype('str')
    encoder = preprocessing.LabelEncoder()
    encoded_list = encoder.fit_transform(df[column])
    #print(encoded_list)
    #print(len(encoded_list))
    encoded_series = pd.Series(encoded_list)
    df[column] = encoded_series
    print("The ", column, "is encoded ")
  return(df)

df = encode_categorical(df,['Label'])

The  Label is encoded 


In [ ]:
X = np.array(df.drop(labels=['Label', 'Query'],axis=1)).reshape(len(df),1,3)
print(len(X))
y = np.array(df['Label'])
print(y)

12887
[1 1 1 ... 0 0 0]


#Split datasets into Test and Train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.2, random_state=42)
train_dataset_df = tf.data.Dataset.from_tensor_slices((X_train_df, y_train_df))
test_dataset_df = tf.data.Dataset.from_tensor_slices((X_test_df, y_test_df))
BATCH_SIZE_df = 64
SHUFFLE_BUFFER_SIZE_df = 100
train_dataset_df = train_dataset_df.shuffle(SHUFFLE_BUFFER_SIZE_df).batch(BATCH_SIZE_df)
test_dataset_df = test_dataset_df.batch(BATCH_SIZE_df)

In [ ]:
#split into test-train datasets
X_trainQuerys, X_testQuerys, y_trainQuerys, y_testQuerys = train_test_split(XQuerys, yQuerys, test_size=0.2, random_state = 0)



```
'''
# Feature Scaling - not used
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
'''
```



#Create ML models for SQL injection

#Using Logistic Regression (LR)

**Logistic Regression (LR)**

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train_df, y_train_df)
y_pred_df = clf.predict(X_test_df)

#Using Simple NN (SEQUENTIAL)

#Generate NN Models

**NN Model-1**

In [ ]:
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Dense(20, activation='relu', input_shape= (None,3)))
model1.add(tf.keras.layers.Dense(16, activation='relu'))
model1.add(tf.keras.layers.Dense(1))

**NN Model-2**

In [ ]:
input_dim = X_train_df.shape[1]  # Number of features
model2 = Sequential()
model2.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model2.add(layers.Dense(10, activation='tanh'))
model2.add(layers.Dense(1024, activation='relu'))
model2.add(layers.BatchNormalization())
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(1, activation='sigmoid')) #1=noOfNeurons



```
#tanh distorts more than sigmoid
# tanh(x) = 2 / (1 + pow(e,-2x)) => S-shaped curve cut at y=1
# sigmoid(x) = 1 / (1 + pow(e,-x)) => S-shaped curve cut at y=0.5
```



#Compile NN Models

**NN Model-1**

In [ ]:
model1.compile(optimizer='adam', metrics=['accuracy'], 
               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))
history = model1.fit(train_dataset, epochs=20, verbose=1)
pred = model1.predict(X_test)
for i in range(len(pred)):
  if pred[i]>0.5:
    pred[i]=1
  elif pred[i]<=0.5:
    pred[i]=0

Epoch 1/20
162/162 [==============================] - 1s 2ms/step - loss: 1.3980 - accuracy: 0.7682
Epoch 2/20
162/162 [==============================] - 0s 2ms/step - loss: 0.1633 - accuracy: 0.9612
Epoch 3/20
162/162 [==============================] - 0s 1ms/step - loss: 0.1347 - accuracy: 0.9983
Epoch 4/20
162/162 [==============================] - 0s 1ms/step - loss: 0.1142 - accuracy: 0.9976
Epoch 5/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0978 - accuracy: 0.9968
Epoch 6/20
162/162 [==============================] - 0s 1ms/step - loss: 0.0850 - accuracy: 0.9955
Epoch 7/20
162/162 [==============================] - 0s 1ms/step - loss: 0.0751 - accuracy: 0.9949
Epoch 8/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0672 - accuracy: 0.9941
Epoch 9/20
162/162 [==============================] - 0s 1ms/step - loss: 0.0610 - accuracy: 0.9940
Epoch 10/20
162/162 [==============================] - 0s 1ms/step - loss: 0.0561 - accuracy: 0.9938

In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, None, 20)          80        
_________________________________________________________________
dense_1 (Dense)              (None, None, 16)          336       
_________________________________________________________________
dense_2 (Dense)              (None, None, 1)           17        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


**NN Model-2**

In [ ]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier_nn = model2.fit(X_train_df,y_train_df, epochs=10, verbose=True, 
                          validation_data=(X_test_df, y_test_df), batch_size=15)
pred = model2.predict(X_test_df)
for i in range(len(pred)):
  if(pred[i] > 0.5):
    pred[i] = 1
  elif(pred[i] <= 0.5):
    pred[i] = 0

Epoch 1/10
224/224 [==============================] - 18s 8ms/step - loss: 0.3319 - accuracy: 0.8480 - val_loss: 0.6853 - val_accuracy: 0.7000
Epoch 2/10
224/224 [==============================] - 1s 4ms/step - loss: 0.1203 - accuracy: 0.9541 - val_loss: 0.3950 - val_accuracy: 0.7083
Epoch 3/10
224/224 [==============================] - 1s 4ms/step - loss: 0.0900 - accuracy: 0.9656 - val_loss: 0.0664 - val_accuracy: 0.9798
Epoch 4/10
224/224 [==============================] - 1s 4ms/step - loss: 0.0879 - accuracy: 0.9741 - val_loss: 0.0599 - val_accuracy: 0.9798
Epoch 5/10
224/224 [==============================] - 1s 4ms/step - loss: 0.0804 - accuracy: 0.9714 - val_loss: 0.0589 - val_accuracy: 0.9786
Epoch 6/10
224/224 [==============================] - 1s 4ms/step - loss: 0.0774 - accuracy: 0.9701 - val_loss: 0.1005 - val_accuracy: 0.9714
Epoch 7/10
224/224 [==============================] - 1s 4ms/step - loss: 0.0818 - accuracy: 0.9685 - val_loss: 0.0804 - val_accuracy: 0.9714
Epoch

In [ ]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                94360     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 110,955
Trainable params: 108,907
Non-trainable params: 2,048
______________________________________________

#Using Gradient Boosting Tree (GBT)

**Gradient Boosting Tree Model**

In [ ]:
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
n_est = [20, 40, 60, 80, 100]
gbt_acc = []

for learning_rate in lr_list: 
  n_est_acc = [] 
  #print("Learning rate: ", learning_rate)
  for n_estimators in n_est:
    gb_clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, 
                                        max_depth=7, random_state=0)
    gb_clf.fit(X_trainQuerys, y_trainQuerys)
    n_est_acc.append(gb_clf.score(X_testQuerys, y_testQuerys))
  gbt_acc.append(n_est_acc)

#Evaluation Metrics

#Generating Appropriate Evaluation Metrics

In [ ]:
def accuracy_function(tp,tn,fp,fn):
  accuracy = (tp+tn) / (tp+tn+fp+fn)
  return accuracy

def precision_function(tp,fp):
  precision = tp / (tp+fp)
  return precision

def recall_function(tp,fn):
  recall=tp / (tp+fn)
  return recall

In [ ]:
def confusion_matrix(truth,predicted):
  true_positive = 0
  true_negative = 0
  false_positive = 0
  false_negative = 0
    
  for true,pred in zip(truth,predicted):
    if true == 1:
      if pred == true:
        true_positive += 1
      elif pred != true:
        false_negative += 1

    elif true == 0:
      if pred == true:
        true_negative+= 1
      elif pred != true:
        false_positive+= 1
            
  accuracy = accuracy_function(true_positive, true_negative, false_positive, false_negative)
  precision = precision_function(true_positive, false_positive)
  recall = recall_function(true_positive, false_negative)  
  return (accuracy, precision, recall)

#Applying Evaluation Metric on the defined ML Models

**Logistic Regression (LR)**

In [ ]:
lr_ac = accuracy_score(y_test_df, y_pred_df)
print("Evaluation metrics for LR Model: \n")
print("Accuracy: ", lr_ac)

Evaluation metrics for LR Model: 

Accuracy:  0.9285714285714286


**NN Model-1**

In [ ]:
model1.evaluate(test_dataset)

41/41 [==============================] - 0s 1ms/step - loss: 0.0525 - accuracy: 0.9938


[0.052461378276348114, 0.9937936663627625]

In [ ]:
accuracy1, precision1, recall1 = confusion_matrix(y_test,pred)
print("Evaluation metrics for NN Model-1: \n")
print("Accuracy: {0} \nPrecision score: {1} \nRecall score: {2}".format(accuracy1, precision1, recall1))

Evaluation metrics for NN Model-1: 

Accuracy: 0.3630952380952381 
Precision score: 0.8185185185185185 
Recall score: 0.3125884016973126


**NN Model-2**

In [ ]:
model2.evaluate(test_dataset_df)

14/14 [==============================] - 0s 5ms/step - loss: 0.0622 - accuracy: 0.9762


[0.062163662165403366, 0.976190447807312]

In [ ]:
accuracy2, precision2, recall2 = confusion_matrix(y_test_df,pred)
print("Evaluation metrics for NN Model-2: \n")
print("Accuracy: {0} \nPrecision score: {1} \nRecall score: {2}".format(accuracy2, precision2, recall2))

Evaluation metrics for NN Model-2: 

Accuracy: 0.9761904761904762 
Precision score: 0.9296296296296296 
Recall score: 0.996031746031746


**Gradient Boosting Tree Model**

In [ ]:
print("Evaluation metrics for GBT Model: \n")

for i_lr in range(0,len(gbt_acc)): 
  print("FOR LEARNING RATE=", lr_list[i_lr], ":--------------- ")
  for i_n_est in range(0,len(n_est)):
    print(i_n_est, ") ", "FOR NO.OF ESTIMATORS=", n_est[i_n_est], ": ")
    print("Accuracy: {0}".format(gbt_acc[i_lr][i_n_est]))
  print("\n")

Evaluation metrics for GBT Model: 

FOR LEARNING RATE= 0.05 :--------------- 
0 )  FOR NO.OF ESTIMATORS= 20 : 
Accuracy: 0.9994805194805195
1 )  FOR NO.OF ESTIMATORS= 40 : 
Accuracy: 0.9994805194805195
2 )  FOR NO.OF ESTIMATORS= 60 : 
Accuracy: 0.9994805194805195
3 )  FOR NO.OF ESTIMATORS= 80 : 
Accuracy: 0.9994805194805195
4 )  FOR NO.OF ESTIMATORS= 100 : 
Accuracy: 0.9994805194805195


FOR LEARNING RATE= 0.075 :--------------- 
0 )  FOR NO.OF ESTIMATORS= 20 : 
Accuracy: 0.9994805194805195
1 )  FOR NO.OF ESTIMATORS= 40 : 
Accuracy: 0.9994805194805195
2 )  FOR NO.OF ESTIMATORS= 60 : 
Accuracy: 0.9994805194805195
3 )  FOR NO.OF ESTIMATORS= 80 : 
Accuracy: 0.9994805194805195
4 )  FOR NO.OF ESTIMATORS= 100 : 
Accuracy: 0.9994805194805195


FOR LEARNING RATE= 0.1 :--------------- 
0 )  FOR NO.OF ESTIMATORS= 20 : 
Accuracy: 0.9994805194805195
1 )  FOR NO.OF ESTIMATORS= 40 : 
Accuracy: 0.9994805194805195
2 )  FOR NO.OF ESTIMATORS= 60 : 
Accuracy: 0.9994805194805195
3 )  FOR NO.OF ESTIMATORS=

In information retrieval,<br> 
Precision_score=1:
means that every result retrieved by a search was relevant 
(but says nothing about whether all relevant documents retrieved) 
<br>
Recall_score=1:
means that all relevant documents were retrieved by the search 
(but says nothing about how many irrelevant documents also retrieved)



---



#Predict for Input Queries

In [ ]:
#check test data using ensemble of GBT
def Check_is_sql(sql):
  # do some pre-processing remoce comment /**/, /*!num */
  _tmp = re.sub(r'(/\*[\w\d(\`|\~|\!|\@|\#|\$|\%|\^|\&|\*|\(|\)|\-|\_|\=|\+|\[|\{|\]|\}|\\|\:|\;|\'|\"|\<|\>|\,|\.|\?)\s\r\n\v\f]*\*/)', 
                ' ', sql)
  _tmp = re.sub(r'(/\*!\d+|\*/)', ' ', _tmp)
  sql_tokens = Sql_tokenizer(_tmp.strip())
  token_seq = GetTokenSeq(sql_tokens, 3)
  sqli_g_means = G_means(token_seq, 'sqli_GTest')
  plain_g_means = G_means(token_seq, 'plain_GTest')
  
  #actually supposed to use ensemble using previously developed GBT models
  gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.75, max_depth=7, random_state=0)
  gb_clf.fit(X_trainQuerys, y_trainQuerys)   
  _X = [[len(sql_tokens), Entropy(sql), sqli_g_means, plain_g_means]]
  return gb_clf.predict(_X)[0]



```
ch1 = '-1923 union select scott, python, machine, learning, study, version, 1--'
ch2 = 'this is a job you dont mean'
ch3 = "%' or '0'='0"
ch4 = "insert into students values('ram','singh',9.9);"
```



In [ ]:
check_data = input('Enter the query to check (SQL_injection/ normal_text): ')
res = Check_is_sql(check_data)
print ("Predicting input query “%s” (SQL_injection/ normal_text):-------->\n" %check_data)

if res == 1:
    print ("The given input query “%s” can be used for SQL_injection. Please restructure your database accordingly...!!!" %check_data)
else:
    print ("The given input query “%s” is normal text. Your database is safe to this query with regards to SQL_injection...." %check_data)

Enter the query to check (SQL_injection/ normal_text): insert into student values 2, 'saharsh','sayak';
Predicting input query “insert into student values 2, 'saharsh','sayak';” (SQL_injection/ normal_text):-------->

The given input query “insert into student values 2, 'saharsh','sayak';” is normal text. Your database is safe to this query with regards to SQL_injection....
